In [354]:
import numpy as np
import pandas as pd
import pymap3d as pm
import pymap3d.vincenty as pmv
import matplotlib.pyplot as plt
import glob as gl
import scipy.optimize
from tqdm.auto import tqdm
from scipy.interpolate import InterpolatedUnivariateSpline
from scipy.spatial import distance
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [355]:
# Constants
CLIGHT = 299_792_458   # speed of light (m/s)
RE_WGS84 = 6_378_137   # earth semimajor axis (WGS84) (m)
OMGE = 7.2921151467E-5  # earth angular velocity (IS-GPS) (rad/s)

In [356]:
# Satellite selection using carrier frequency error, elevation angle, and C/N0
def satellite_selection(df, column):
    """
    Args:
        df : DataFrame from device_gnss.csv
        column : Column name
    Returns:
        df: DataFrame with eliminated satellite signals
    """
    idx = df[column].notnull()
    idx &= df['CarrierErrorHz'] < 2.0e6  # carrier frequency error (Hz)
    idx &= df['SvElevationDegrees'] > 10.0  # elevation angle (deg)
    idx &= df['Cn0DbHz'] > 15.0  # C/N0 (dB-Hz)
    idx &= df['MultipathIndicator'] == 0 # Multipath flag

    return df[idx]

In [357]:
# Compute line-of-sight vector from user to satellite
def los_vector(xusr, xsat):
    """
    Args:
        xusr : user position in ECEF (m)
        xsat : satellite position in ECEF (m)
    Returns:
        u: unit line-of-sight vector in ECEF (m)
        rng: distance between user and satellite (m)
    """
    u = xsat - xusr
    rng = np.linalg.norm(u, axis=1).reshape(-1, 1)
    u /= rng
    
    return u, rng.reshape(-1)


# Compute Jacobian matrix
def jac_pr_residuals(x, xsat, pr, W):
    """
    Args:
        x : current position in ECEF (m)
        xsat : satellite position in ECEF (m)
        pr : pseudorange (m)
        W : weight matrix
    Returns:
        W*J : Jacobian matrix
    """
    u, _ = los_vector(x[:3], xsat)
    J = np.hstack([-u, np.ones([len(pr), 1])])  # J = [-ux -uy -uz 1]

    return W @ J


# Compute pseudorange residuals
def pr_residuals(x, xsat, pr, W):
    """
    Args:
        x : current position in ECEF (m)
        xsat : satellite position in ECEF (m)
        pr : pseudorange (m)
        W : weight matrix
    Returns:
        residuals*W : pseudorange residuals
    """
    u, rng = los_vector(x[:3], xsat)

    # Approximate correction of the earth rotation (Sagnac effect) often used in GNSS positioning
    rng += OMGE * (xsat[:, 0] * x[1] - xsat[:, 1] * x[0]) / CLIGHT

    # Add GPS L1 clock offset
    residuals = rng - (pr - x[3])

    return residuals @ W


# Compute Jacobian matrix
def jac_prr_residuals(v, vsat, prr, x, xsat, W):
    """
    Args:
        v : current velocity in ECEF (m/s)
        vsat : satellite velocity in ECEF (m/s)
        prr : pseudorange rate (m/s)
        x : current position in ECEF (m)
        xsat : satellite position in ECEF (m)
        W : weight matrix
    Returns:
        W*J : Jacobian matrix
    """
    u, _ = los_vector(x[:3], xsat)
    J = np.hstack([-u, np.ones([len(prr), 1])])

    return np.dot(W, J)


# Compute pseudorange rate residuals
def prr_residuals(v, vsat, prr, x, xsat, W):
    """
    Args:
        v : current velocity in ECEF (m/s)
        vsat : satellite velocity in ECEF (m/s)
        prr : pseudorange rate (m/s)
        x : current position in ECEF (m)
        xsat : satellite position in ECEF (m)
        W : weight matrix
    Returns:
        residuals*W : pseudorange rate residuals
    """
    u, rng = los_vector(x[:3], xsat)
    rate = np.sum((vsat-v[:3])*u, axis=1) \
          + OMGE / CLIGHT * (vsat[:, 1] * x[0] + xsat[:, 1] * v[0]
                           - vsat[:, 0] * x[1] - xsat[:, 0] * v[1])

    residuals = rate - (prr - v[3])

    return residuals @ W

In [358]:
# Carrier smoothing of pseudarange
def carrier_smoothing(gnss_df):
    """
    Args:
        df : DataFrame from device_gnss.csv
    Returns:
        df: DataFrame with carrier-smoothing pseudorange 'pr_smooth'
    """
    carr_th = 1.6 # carrier phase jump threshold [m] 2->1.5 (best)->1.0
    pr_th =  20.0 # pseudorange jump threshold [m] 20->15

    prsmooth = np.full_like(gnss_df['RawPseudorangeMeters'], np.nan)
    # Loop for each signal
    for (i, (svid_sigtype, df)) in enumerate((gnss_df.groupby(['Svid', 'SignalType']))):
        df = df.replace(
            {'AccumulatedDeltaRangeMeters': {0: np.nan}})  # 0 to NaN

        # Compare time difference between pseudorange/carrier with Doppler
        drng1 = df['AccumulatedDeltaRangeMeters'].diff() - df['PseudorangeRateMetersPerSecond']
        drng2 = df['RawPseudorangeMeters'].diff() - df['PseudorangeRateMetersPerSecond']

        # Check cycle-slip
        slip1 = (df['AccumulatedDeltaRangeState'].to_numpy() & 2**1) != 0  # reset flag
        slip2 = (df['AccumulatedDeltaRangeState'].to_numpy() & 2**2) != 0  # cycle-slip flag
        slip3 = np.fabs(drng1.to_numpy()) > carr_th # Carrier phase jump
        slip4 = np.fabs(drng2.to_numpy()) > pr_th # Pseudorange jump

        idx_slip = slip1 | slip2 | slip3 | slip4
        idx_slip[0] = True

        # groups with continuous carrier phase tracking
        df['group_slip'] = np.cumsum(idx_slip)

        # Psudorange - carrier phase
        df['dpc'] = df['RawPseudorangeMeters'] - df['AccumulatedDeltaRangeMeters']

        # Absolute distance bias of carrier phase
        meandpc = df.groupby('group_slip')['dpc'].mean()
        df = df.merge(meandpc, on='group_slip', suffixes=('', '_Mean'))

        # Index of original gnss_df
        idx = (gnss_df['Svid'] == svid_sigtype[0]) & (
            gnss_df['SignalType'] == svid_sigtype[1])

        # Carrier phase + bias
        prsmooth[idx] = df['AccumulatedDeltaRangeMeters'] + df['dpc_Mean']

    # If carrier smoothing is not possible, use original pseudorange
    idx_nan = np.isnan(prsmooth)
    prsmooth[idx_nan] = gnss_df['RawPseudorangeMeters'][idx_nan]
    gnss_df['pr_smooth'] = prsmooth

    return gnss_df

In [359]:
# Compute distance by Vincenty's formulae
def vincenty_distance(llh1, llh2):
    """
    Args:
        llh1 : [latitude,longitude] (deg)
        llh2 : [latitude,longitude] (deg)
    Returns:
        d : distance between llh1 and llh2 (m)
    """
    d, az = np.array(pmv.vdist(llh1[:, 0], llh1[:, 1], llh2[:, 0], llh2[:, 1]))

    return d


# Compute score
def calc_score(llh, llh_gt):
    """
    Args:
        llh : [latitude,longitude] (deg)
        llh_gt : [latitude,longitude] (deg)
    Returns:
        score : (m)
    """
    d = vincenty_distance(llh, llh_gt)
    score = np.mean([np.quantile(d, 0.50), np.quantile(d, 0.95)])

    return score

In [360]:
# GNSS single point positioning using pseudorange

def point_positioning(gnss_df):
    # Add nominal frequency to each signal
    # Note: GLONASS is an FDMA signal, so each satellite has a different frequency
    CarrierFrequencyHzRef = gnss_df.groupby(['Svid', 'SignalType'])[
        'CarrierFrequencyHz'].median()
    gnss_df = gnss_df.merge(CarrierFrequencyHzRef, how='left', on=[
                            'Svid', 'SignalType'], suffixes=('', 'Ref'))
    gnss_df['CarrierErrorHz'] = np.abs(
        (gnss_df['CarrierFrequencyHz'] - gnss_df['CarrierFrequencyHzRef']))

    # Carrier smoothing
    gnss_df = carrier_smoothing(gnss_df)

    # GNSS single point positioning
    utcTimeMillis = gnss_df['utcTimeMillis'].unique()
    nepoch = len(utcTimeMillis)
    x0 = np.zeros(4)  # [x,y,z,tGPSL1]
    v0 = np.zeros(4)  # [vx,vy,vz,dtGPSL1]
    x_wls = np.full([nepoch, 3], np.nan)  # For saving position
    v_wls = np.full([nepoch, 3], np.nan)  # For saving velocity

    # Loop for epochs
    for i, (t_utc, df) in enumerate(tqdm(gnss_df.groupby('utcTimeMillis'), total=nepoch)):
        # Valid satellite selection
        df_pr = satellite_selection(df, 'pr_smooth')
        df_prr = satellite_selection(df, 'PseudorangeRateMetersPerSecond')

        # Corrected pseudorange/pseudorange rate
        pr = (df_pr['pr_smooth'] + df_pr['SvClockBiasMeters'] - df_pr['IsrbMeters'] -
              df_pr['IonosphericDelayMeters'] - df_pr['TroposphericDelayMeters']).to_numpy()
        prr = (df_prr['PseudorangeRateMetersPerSecond'] +
               df_prr['SvClockDriftMetersPerSecond']).to_numpy()

        # Satellite position/velocity
        xsat_pr = df_pr[['SvPositionXEcefMeters', 'SvPositionYEcefMeters',
                         'SvPositionZEcefMeters']].to_numpy()
        xsat_prr = df_prr[['SvPositionXEcefMeters', 'SvPositionYEcefMeters',
                           'SvPositionZEcefMeters']].to_numpy()
        vsat = df_prr[['SvVelocityXEcefMetersPerSecond', 'SvVelocityYEcefMetersPerSecond',
                       'SvVelocityZEcefMetersPerSecond']].to_numpy()

        # Weight matrix for peseudorange/pseudorange rate
        Wx = np.diag(1 / df_pr['RawPseudorangeUncertaintyMeters'].to_numpy())
        Wv = np.diag(1 / df_prr['PseudorangeRateUncertaintyMetersPerSecond'].to_numpy())

        # Robust WLS requires accurate initial values for convergence,
        # so perform normal WLS for the first time
        if len(df_pr) >= 4:
            # Normal WLS
            if np.all(x0 == 0):
                opt = scipy.optimize.least_squares(
                    pr_residuals, x0, jac_pr_residuals, args=(xsat_pr, pr, Wx))
                x0 = opt.x 
            # Robust WLS for position estimation
            opt = scipy.optimize.least_squares(
                 pr_residuals, x0, jac_pr_residuals, args=(xsat_pr, pr, Wx), loss='soft_l1')
            if opt.status < 1 or opt.status == 2:
                 print(f'i = {i} position lsq status = {opt.status}')
            else:
                 x_wls[i, :] = opt.x[:3]
                 x0 = opt.x
                 
        # Velocity estimation
        if len(df_prr) >= 4:
            if np.all(v0 == 0): # Normal WLS
                opt = scipy.optimize.least_squares(
                    prr_residuals, v0, jac_prr_residuals, args=(vsat, prr, x0, xsat_prr, Wv))
                v0 = opt.x
            # Robust WLS for velocity estimation
            opt = scipy.optimize.least_squares(
                prr_residuals, v0, jac_prr_residuals, args=(vsat, prr, x0, xsat_prr, Wv), loss='soft_l1')
            if opt.status < 1:
                print(f'i = {i} velocity lsq status = {opt.status}')
            else:
                v_wls[i, :] = opt.x[:3]
                v0 = opt.x

    return utcTimeMillis, x_wls, v_wls

In [361]:
# Simple outlier detection and interpolation
def exclude_interpolate_outlier(x_wls, v_wls):
    # Up velocity threshold
    v_up_th = 2.0 # m/s

    # Coordinate conversion
    x_llh = np.array(pm.ecef2geodetic(x_wls[:, 0], x_wls[:, 1], x_wls[:, 2])).T
    v_enu = np.array(pm.ecef2enuv(
        v_wls[:, 0], v_wls[:, 1], v_wls[:, 2], x_llh[0, 0], x_llh[0, 1])).T

    # Up velocity jump detection
    # Cars don't jump suddenly!
    idx_v_out = np.abs(v_enu[:, 2]) > v_up_th
    v_wls[idx_v_out, :] = np.nan
    
    # Interpolate NaNs at beginning and end of array
    x_df = pd.DataFrame({'x': x_wls[:, 0], 'y': x_wls[:, 1], 'z': x_wls[:, 2]})
    x_df = x_df.interpolate(limit_area='outside', limit_direction='both')
    
    # Interpolate all NaN data
    v_df = pd.DataFrame({'x': v_wls[:, 0], 'y': v_wls[:, 1], 'z': v_wls[:, 2]})
    v_df = v_df.interpolate(limit_area='outside', limit_direction='both')
    v_df = v_df.interpolate('spline', order=3)

    return x_df.to_numpy(), v_df.to_numpy()

In [362]:
# Kalman filter
def Kalman_filter(zs, us, phone):
    # Parameters
    # I don't know why only XiaomiMi8 seems to be inaccurate ... 
    sigma_v = 0.6 if phone == 'XiaomiMi8' else 0.1 # velocity SD m/s
    sigma_x = 5.0  # position SD m
    sigma_mahalanobis = 30.0 # Mahalanobis distance for rejecting innovation
    
    n, dim_x = zs.shape
    F = np.eye(3)  # Transition matrix
    Q = sigma_v**2 * np.eye(3)  # Process noise

    H = np.eye(3)  # Measurement function
    R = sigma_x**2 * np.eye(3)  # Measurement noise

    # Initial state and covariance
    x = zs[0, :3].T  # State
    P = sigma_x**2 * np.eye(3)  # State covariance
    I = np.eye(dim_x)

    x_kf = np.zeros([n, dim_x])
    P_kf = np.zeros([n, dim_x, dim_x])

    # Kalman filtering
    for i, (u, z) in enumerate(zip(us, zs)):
        # First step
        if i == 0:
            x_kf[i] = x.T
            P_kf[i] = P
            continue

        # Prediction step
        x = F @ x + u.T
        P = (F @ P) @ F.T + Q

        # Check outliers for observation
        d = distance.mahalanobis(z, H @ x, np.linalg.pinv(P))

        # Update step
        if d < sigma_mahalanobis:
            y = z.T - H @ x
            S = (H @ P) @ H.T + R
            K = (P @ H.T) @ np.linalg.inv(S)
            x = x + K @ y
            P = (I - (K @ H)) @ P
        else:
            # If no observation update is available, increase covariance
            P += 10**2*Q

        x_kf[i] = x.T
        P_kf[i] = P

    return x_kf, P_kf


# Forward + backward Kalman filter and smoothing
def Kalman_smoothing(x_wls, v_wls, phone):
    n, dim_x = x_wls.shape

    # Forward
    v = np.vstack([np.zeros([1, 3]), (v_wls[:-1, :] + v_wls[1:, :])/2])
    x_f, P_f = Kalman_filter(x_wls, v, phone)

    # Backward
    v = -np.flipud(v_wls)
    v = np.vstack([np.zeros([1, 3]), (v[:-1, :] + v[1:, :])/2])
    x_b, P_b = Kalman_filter(np.flipud(x_wls), v, phone)

    # Smoothing
    x_fb = np.zeros_like(x_f)
    P_fb = np.zeros_like(P_f)
    for (f, b) in zip(range(n), range(n-1, -1, -1)):
        P_fi = np.linalg.inv(P_f[f])
        P_bi = np.linalg.inv(P_b[b])

        P_fb[f] = np.linalg.inv(P_fi + P_bi)
        x_fb[f] = P_fb[f] @ (P_fi @ x_f[f] + P_bi @ x_b[b])

    return x_fb, x_f, np.flipud(x_b)

In [363]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch
import numpy as np
from sklearn.preprocessing import StandardScaler
from torch import nn

def create_sequences(data, target, sequence_length):
    data_sequences = []
    target_sequences = []
    for i in range(len(data) - sequence_length):
        data_sequences.append(data[i:i+sequence_length])
        target_sequences.append(target[i+sequence_length])
    
    return np.array(data_sequences), np.array(target_sequences)
# data_sequences, target_sequences = create_sequences(data_train, data_target, sequence_length)

In [364]:
# 定义 LSTM 模型
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout_rate):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        # 定义带dropout的LSTM层
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout_rate)
        
        # 定义输出层
        self.linear = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # 初始化隐藏状态和细胞状态
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        
        # 从LSTM层获取所有隐藏状态
        out, _ = self.lstm(x, (h0, c0))
        
        # 取最后一个时间步的输出
        out = self.linear(out[:, -1, :])
        return out

In [365]:
# 训练函数
def train_file(model_train, train_tensor,target_tensor, optimizer, criterion, epochs):
    # loss_history = []  # 用于存储每个epoch的loss，以便绘图
    final_outputs = []
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model_train(train_tensor)
        loss = criterion(outputs, target_tensor)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model_train.parameters(), max_norm=1.0)  # 梯度裁剪
        optimizer.step()
        # scheduler.step(loss)
        
        # loss_history.append(loss.item())  # 记录当前epoch的loss
        if (epoch + 1) % 20 == 0:  # 每10个epoch输出一次
            current_lr = optimizer.param_groups[0]['lr']
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Current LR: {current_lr}')
        if epoch == epochs - 1:
            final_outputs = outputs.detach().numpy()    
        
    return final_outputs

In [366]:
from torch.utils.data import TensorDataset, DataLoader

train_score_bl = []
train_score_wls = []
train_score_kf = []
train_score_lstm = []

def process_array(x_wls):
    try:
        # 检查数据中是否有nan
        if np.any(np.isnan(x_wls)):
            raise ValueError("Data contains NaN values")

        # 进行数据训练或其他处理
        train_score_bl,train_score_wls,train_score_kf,train_score_lstm = train_lstm(x_wls)
        return train_score_bl,train_score_wls,train_score_kf,train_score_lstm

    except Exception as e:
        print("Error processing")
        return None
    
def train_lstm(x_wls):
    # Kalman smoothing
    x_kf, _, _ = Kalman_smoothing(x_wls, v_wls, phone)

    # Baseline
    x_bl = gnss_df.groupby('TimeNanos')[['WlsPositionXEcefMeters', 'WlsPositionYEcefMeters',   'WlsPositionZEcefMeters']].mean().to_numpy()
    llh_bl = np.array(pm.ecef2geodetic(x_bl[:, 0], x_bl[:, 1], x_bl[:, 2])).T
    
    # data_diff = x_kf - x_bl
    utc_array = np.array(utc).reshape(-1,1)
    data_train = np.hstack((x_kf,utc_array))
    data_target =  x_bl
    train_sequence , target_sequence = create_sequences(data_train,data_target,sequence_length)
    train_tensor = torch.tensor(train_sequence, dtype=torch.float32)
    target_tensor = torch.tensor(target_sequence, dtype=torch.float32)

    x_lstm = train_file(model_train, train_tensor,target_tensor,optimizer,loss_function,epochs=100)
    x_lstm_final = np.concatenate((x_kf[:sequence_length],x_lstm))

    array1 = x_kf - x_bl
    array2 = x_lstm_final - x_bl
    x_min = np.where(np.abs(array1) <= np.abs(array2), array1, array2)
    x_min_final = x_min + x_bl
    
    # compute score
    llh_gt = gt_df[['LatitudeDegrees', 'LongitudeDegrees']].to_numpy()
    llh_wls = np.array(pm.ecef2geodetic(x_wls[:, 0], x_wls[:, 1], x_wls[:, 2])).T
    llh_kf = np.array(pm.ecef2geodetic(x_kf[:, 0], x_kf[:, 1], x_kf[:, 2])).T
    llh_lstm = np.array(pm.ecef2geodetic(x_min_final[:, 0], x_min_final[:, 1],x_min_final[:, 2])).T
    
    # Distance from ground truth
    len_diff = llh_bl.shape[0] - llh_gt.shape[0]
    vd_bl = vincenty_distance(llh_bl[len_diff:], llh_gt)
    vd_wls = vincenty_distance(llh_wls[len_diff:], llh_gt)
    vd_kf = vincenty_distance(llh_kf[len_diff:], llh_gt)
    vd_lstm = vincenty_distance(llh_lstm[len_diff:], llh_gt)
    
    
    score_bl = calc_score(llh_bl[len_diff:], llh_gt)
    train_score_bl.append(score_bl)
    score_wls = calc_score(llh_wls[len_diff:], llh_gt)
    train_score_wls.append(score_wls)
    score_kf = calc_score(llh_kf[len_diff:-1, :], llh_gt[:-1, :])
    train_score_kf.append(score_kf)
    score_lstm = calc_score(llh_lstm[len_diff:, :], llh_gt[:, :])
    train_score_lstm.append(score_lstm)
    
    return train_score_bl,train_score_wls,train_score_kf,train_score_lstm

In [368]:
# Loop for each trip
path = 'D:/PythonProject/FinalProject/smartphone-decimeter-2023/sdc2023/train'

num_SuccessFile = 0 
num_features = 4
hidden_dim = 64  # LSTM隐藏层维数
output_dim = 3  # 输出类别数
num_layers = 3  # LSTM层数
dropout = 0.5
sequence_length = 10
# 实例化模型
model_train = LSTMModel(num_features, hidden_dim, output_dim, num_layers, dropout)
optimizer = torch.optim.Adam(model_train.parameters(), lr=0.001)
loss_function = nn.MSELoss()

for i, dirname in enumerate(tqdm(sorted(gl.glob(f'{path}/*/*')))):
    cleaned_path = dirname.replace('\\', '/').rstrip('/')
    
    # 分割路径并选择最后两个部分作为 drive 和 phone
    drive, phone = cleaned_path.split('/')[-2:]
    
    # 输出 drive 和 phone 以验证
    print(f"Drive: {drive}, Phone: {phone}")

    # Read data
    gnss_df = pd.read_csv(f'{dirname}/device_gnss.csv',low_memory=False)
    gt_df = pd.read_csv(f'{dirname}/ground_truth.csv',low_memory=False)
    
    # Point positioning
    utc, x_wls, v_wls = point_positioning(gnss_df)

    # Exclude velocity outliers
    x_wls, v_wls = exclude_interpolate_outlier(x_wls, v_wls)
    
    checkpoint = process_array(x_wls)
    if checkpoint is not None:
        num_SuccessFile = num_SuccessFile + 1
        print("Processed successfully %d/%d."%(num_SuccessFile,i+1))
    else:
        print("Skipped due to errors.")

  0%|          | 0/156 [00:00<?, ?it/s]

Drive: 2020-06-25-00-34-us-ca-mtv-sb-101, Phone: pixel4


  0%|          | 0/1300 [00:00<?, ?it/s]

Epoch [20/100], Loss: 13526596845568.0000, Current LR: 0.001
Epoch [40/100], Loss: 13526579019776.0000, Current LR: 0.001
Epoch [60/100], Loss: 13526567485440.0000, Current LR: 0.001
Epoch [80/100], Loss: 13526559096832.0000, Current LR: 0.001
Epoch [100/100], Loss: 13526548611072.0000, Current LR: 0.001
Processed successfully 1/1.
Drive: 2020-06-25-00-34-us-ca-mtv-sb-101, Phone: pixel4xl


  0%|          | 0/1303 [00:00<?, ?it/s]

...

i = 69 position lsq status = 2
i = 75 position lsq status = 2
i = 76 position lsq status = 2
i = 78 position lsq status = 2
i = 80 position lsq status = 2
i = 82 position lsq status = 2
i = 88 position lsq status = 2
i = 89 position lsq status = 2
i = 99 position lsq status = 2
i = 100 position lsq status = 2
i = 823 position lsq status = 2
i = 826 position lsq status = 2
i = 827 position lsq status = 2
i = 828 position lsq status = 2
i = 831 position lsq status = 2
i = 832 position lsq status = 2
i = 834 position lsq status = 2
i = 836 position lsq status = 2
i = 838 position lsq status = 2
i = 850 position lsq status = 2
i = 865 position lsq status = 2
i = 866 position lsq status = 2
i = 869 position lsq status = 2
i = 872 position lsq status = 2
i = 883 position lsq status = 2
i = 884 position lsq status = 2
i = 885 position lsq status = 2
i = 900 position lsq status = 2
i = 905 position lsq status = 2
i = 906 position lsq status = 2
i = 908 position lsq status = 2
i = 919 position 

  0%|          | 0/2070 [00:00<?, ?it/s]

Epoch [20/100], Loss: 13521670635520.0000, Current LR: 0.001
Epoch [40/100], Loss: 13521660149760.0000, Current LR: 0.001
Epoch [60/100], Loss: 13521650712576.0000, Current LR: 0.001
Epoch [80/100], Loss: 13521641275392.0000, Current LR: 0.001
Epoch [100/100], Loss: 13521631838208.0000, Current LR: 0.001
Processed successfully 109/155.
Drive: 2023-09-07-22-48-us-ca-routebc2, Phone: pixel5a


  0%|          | 0/2072 [00:00<?, ?it/s]

Epoch [20/100], Loss: 13521627643904.0000, Current LR: 0.001
Epoch [40/100], Loss: 13521618206720.0000, Current LR: 0.001
Epoch [60/100], Loss: 13521610866688.0000, Current LR: 0.001
Epoch [80/100], Loss: 13521600380928.0000, Current LR: 0.001
Epoch [100/100], Loss: 13521591992320.0000, Current LR: 0.001
Processed successfully 110/156.


In [347]:
# torch.save(model_train.state_dict(), 'model_state_dict.pth')

In [370]:
print(np.nanmean(train_score_bl))
print(np.nanmean(train_score_wls))
print(np.nanmean(train_score_kf))
print(np.nanmean(train_score_lstm))

57.465833828051544
56.508961047816676
56.16506888577534
56.81702950366018


65.01834181677802
63.84216011999472
63.42446419834961
64.30112325220965

In [353]:
path = 'D:/PythonProject/FinalProject/smartphone-decimeter-2023/sdc2023/'
sample_df = pd.read_csv(f'{path}/sample_submission.csv')
test_dfs = []
sequence_length = 10
# Loop for each trip
for i, dirname in enumerate(tqdm(sorted(gl.glob(f'{path}/test/*/*/')))):
    drive, phone = dirname.split('/')[-3:-1]
    tripID = f'{drive}/{phone}'
    print(tripID)

    # Read data
    gnss_df = pd.read_csv(f'{dirname}/device_gnss.csv')

    # Point positioning
    utc, x_wls, v_wls = point_positioning(gnss_df)
    utc_array = np.array(utc).reshape(-1,1)

    # Exclude velocity outliers
    x_wls, v_wls = exclude_interpolate_outlier(x_wls, v_wls)

    # Kalman smoothing
    x_kf, _, _ = Kalman_smoothing(x_wls, v_wls, phone)

    # Convert to latitude and longitude
    # llh_kf = np.array(pm.ecef2geodetic(x_kf[:, 0], x_kf[:, 1], x_kf[:, 2])).T
    x_test = np.hstack((x_kf, utc_array)) 
    test_seq = []    
    for i in range(len(x_test) - sequence_length):
        test_seq.append(x_test[i:i+sequence_length])
    X_pred_tensor = torch.tensor(test_seq, dtype=torch.float32)
    model_train.eval()  # 设置模型为评估模式
    with torch.no_grad():
        x_lstm = model_train(X_pred_tensor)  # 假设 test_data 已准备好
        predictions = np.concatenate((x_kf[:sequence_length],x_lstm))

    llh_kf = np.array(pm.ecef2geodetic(predictions[:, 0], predictions[:, 1], predictions[:, 2])).T
    # Interpolation for submission
    UnixTimeMillis = sample_df[sample_df['tripId'] == tripID]['UnixTimeMillis'].to_numpy()
    lat = InterpolatedUnivariateSpline(utc, llh_kf[:,0], ext=3)(UnixTimeMillis)
    lng = InterpolatedUnivariateSpline(utc, llh_kf[:,1], ext=3)(UnixTimeMillis)
    trip_df = pd.DataFrame({
        'tripId' : tripID,
        'UnixTimeMillis': UnixTimeMillis,
        'LatitudeDegrees': lat,
        'LongitudeDegrees': lng
        })

    test_dfs.append(trip_df)

# Write submission.csv
test_df = pd.concat(test_dfs)
test_df.to_csv('submission.csv', index=False)

  0%|          | 0/40 [00:00<?, ?it/s]

sdc2023/


  0%|          | 0/1190 [00:00<?, ?it/s]

C:\Users\Administrator\AppData\Local\Temp\ipykernel_36740\445851347.py:30: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:278.)
  X_pred_tensor = torch.tensor(test_seq, dtype=torch.float32)


sdc2023/


...

  0%|          | 0/1608 [00:00<?, ?it/s]

sdc2023/


C:\Users\Administrator\AppData\Local\Temp\ipykernel_36740\445851347.py:12: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  gnss_df = pd.read_csv(f'{dirname}/device_gnss.csv')


  0%|          | 0/1495 [00:00<?, ?it/s]

i = 628 position lsq status = 2
i = 633 position lsq status = 2
i = 647 position lsq status = 2
